In [3]:
library(assertthat)
library(dplyr)
library(purrr)
library(ggraph)
library(ggmap)
library(ggplot2)
library(igraph)
library(rjson)


Attaching package: 'dplyr'


The following objects are masked from 'package:stats':

    filter, lag


The following objects are masked from 'package:base':

    intersect, setdiff, setequal, union


Loading required package: ggplot2

Google's Terms of Service: https://cloud.google.com/maps-platform/terms/.

Please cite ggmap if you use it! See citation("ggmap") for details.


Attaching package: 'igraph'


The following objects are masked from 'package:purrr':

    compose, simplify


The following objects are masked from 'package:dplyr':

    as_data_frame, groups, union


The following objects are masked from 'package:stats':

    decompose, spectrum


The following object is masked from 'package:base':

    union




## Read in Traffic Data for 4th Quarter Winter, December

In [2]:
santa_filename <- "los_angeles-censustracts-2019-4-All-MonthlyAggregate.csv"
filepath <- "santa_data"
santa_df <- read.csv(file.path(filepath, santa_filename),stringsAsFactors=FALSE,header=TRUE)

In [3]:
geo_filename <- "los_angeles_censustracts.json"
geodata_json <- fromJSON(file=file.path(filepath, geo_filename))

In [4]:
edge_list <- santa_df[santa_df$month==12,c("sourceid","dstid","mean_travel_time")]
names(edge_list) <- c("src","dst","weight")

In [5]:
location_graph <- graph_from_data_frame(edge_list,directed=FALSE)

## Get the Addresses and Longitude and Latitude

In [6]:
V(location_graph)$centroid_x <- NA
V(location_graph)$centroid_y <- NA
V(location_graph)$location_id <- NA

In [7]:
key <- names(read.delim('C:/Users/lpott/Desktop/geolocation_key.txt'))
register_google(key = key)

Warning message in read.table(file = file, header = header, sep = sep, quote = quote, :
"incomplete final line found by readTableHeader on 'C:/Users/lpott/Desktop/geolocation_key.txt'"


In [9]:
for (feature in geodata_json$features){
    centroid <- colMeans(matrix(unlist(feature$geometry$coordinates),ncol=2,byrow=TRUE))
    vertex_id <- feature$properties$MOVEMENT_ID

    if (!(vertex_id %in% V(location_graph)$name)){
        next
    }
    #set.vertex.attribute(location_graph,'centroid',vertex_id,centroid)
    V(location_graph)[vertex_id]$location_id <- revgeocode(c(centroid[1],centroid[2]),output='address')
    V(location_graph)[vertex_id]$centroid_x <- centroid[1]
    V(location_graph)[vertex_id]$centroid_y <- centroid[2]
}

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.1030955737705,-118.120533213115&key=xxx

Multiple addresses found, the first will be returned:

  405 N Granada Ave, Alhambra, CA 91801, USA

  823 E Grand Ave, Alhambra, CA 91801, USA

  400-420 N Granada Ave, Alhambra, CA 91801, USA

  Alhambra, CA 91801, USA

  Alhambra, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  4V3H+6Q Alhambra, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0964512105263,-118.137850631579&key=xxx

Multiple addresses found, the first will be returned:

  400 N Marguerita Ave, Alhambra, CA 91801, USA

  330 N Marguerita Ave, Alhambra, CA 91801, USA

  599-303 N Marguerita Ave, Alhambra, CA 91801, USA

  Alhambra, CA 91801, USA

  Alhambra, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  3VW6+HV Alhambra, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0962638636364,-118.131

  1998-1862 S Stoneman Ave, Alhambra, CA 91801, USA

  Alhambra, CA 91801, USA

  Alhambra, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  3VCH+R6 Alhambra, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0765908695652,-118.126370086957&key=xxx

Multiple addresses found, the first will be returned:

  1510 S 4th St, Alhambra, CA 91803, USA

  1514 S 4th St, Alhambra, CA 91803, USA

  1515 S 3rd St, Alhambra, CA 91803, USA

  1401-1599 S 4th St, Alhambra, CA 91803, USA

  Alhambra, CA 91803, USA

  Alhambra, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  3VGF+JF Alhambra, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0637109428571,-118.142371409524&key=xxx

Multiple addresses found, the first will be returned:

  2520 Snead Dr, Alhambra, CA 91803, USA

  2698-2412 Snead Dr, Alhambra, CA 91803, USA

  Alhambra, CA 91803, USA

  Alhambra, CA, USA

  Los Angeles Count

  9964 Garibaldi Ave, Temple City, CA 91780, USA

  9960 Garibaldi Ave, Temple City, CA 91780, USA

  9975-9965 Garibaldi Ave, Temple City, CA 91780, USA

  Temple City, CA 91780, USA

  Arcadia, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  4W6W+GG Arcadia, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0954319230769,-117.998252092308&key=xxx

Multiple addresses found, the first will be returned:

  5358 Durfee Ave, El Monte, CA 91732, USA

  5340 Durfee Ave, El Monte, CA 91732, USA

  5318 Durfee Ave, El Monte, CA 91732, USA

  5399-5377 Durfee Ave, El Monte, CA 91732, USA

  Arcadia, CA 91006, USA

  Arcadia, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  32W2+5M Arcadia, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.8769025151515,-118.083503727273&key=xxx

Multiple addresses found, the first will be returned:

  16905 Pioneer Blvd, Artesia, CA 90701, USA

  

Multiple addresses found, the first will be returned:

  504 E Renwick Rd, Azusa, CA 91702, USA

  400-502 E Renwick Rd, Azusa, CA 91702, USA

  Azusa, CA, USA

  Azusa, CA 91702, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  436X+83 Azusa, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0720847297297,-117.995355959459&key=xxx

Multiple addresses found, the first will be returned:

  3388 Barnes Ave, Baldwin Park, CA 91706, USA

  3383 Barnes Ave, Baldwin Park, CA 91706, USA

  3399-3359 Barnes Ave, Baldwin Park, CA 91706, USA

  Baldwin Park, CA, USA

  Baldwin Park, CA 91706, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  32C3+RV Baldwin Park, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0705006896552,-117.986603396552&key=xxx

Multiple addresses found, the first will be returned:

  3258 Frazier St, Baldwin Park, CA 91706, USA

  3250 Frazier St, Baldwin Park, CA 917

  Baldwin Park, CA, USA

  Baldwin Park, CA 91706, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  327M+82 Baldwin Park, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0637955616438,-117.976299150685&key=xxx

Multiple addresses found, the first will be returned:

  1249 Stichman Ave, Bassett, CA 91746, USA

  1244-1298 Stichman Ave, Bassett, CA 91746, USA

  Bassett, City of Industry, CA, USA

  La Puente, CA 91746, USA

  Baldwin Park, CA, USA

  Los Angeles County, CA, USA

  California, USA

  United States

  327F+GF Baldwin Park, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=34.0591321666667,-118.003159166667&key=xxx

Multiple addresses found, the first will be returned:

  345 Cloverleaf Dr, La Puente, CA 91746, USA

  365 Cloverleaf Dr, Baldwin Park, CA 91706, USA

  365 Cloverleaf Dr, Bassett, CA 91746, USA

  Bassett, City of Industry, CA, USA

  La Puente, CA 91746, USA

  Baldwin Park, CA, 

...

Multiple addresses found, the first will be returned:

  Perimeter Rd, Huntington Beach, CA 92649, USA

  HUNTINGTN BCH, CA 92649, USA

  Seal Beach, CA, USA

  Orange County, CA, USA

  California, USA

  United States

  PWMW+29 Seal Beach, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.7427983529412,-118.03158704902&key=xxx

Multiple addresses found, the first will be returned:

  5561 Engineer Dr, Huntington Beach, CA 92649, USA

  5542 Engineer Dr, Huntington Beach, CA 92649, USA

  5558-5726 Engineer Dr, Huntington Beach, CA 92649, USA

  HUNTINGTN BCH, CA 92649, USA

  Huntington Beach, CA, USA

  Orange County, CA, USA

  California, USA

  United States

  PXV9+49 Huntington Beach, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.9426731117318,-117.965639553073&key=xxx

Multiple addresses found, the first will be returned:

  1171 Hacienda Rd, La Habra, CA 90631, USA

  1798-1740 Lindauer Dr, La Habra, CA 90631, USA



Multiple addresses found, the first will be returned:

  8241 Sparrow Ln, La Palma, CA 90623, USA

  8251 Sparrow Ln, La Palma, CA 90623, USA

  8221 Moody St, La Palma, CA 90623, USA

  8201-8277 Sparrow Ln, La Palma, CA 90623, USA

  La Palma, CA, USA

  La Palma, CA 90623, USA

  Orange County, CA, USA

  California, USA

  United States

  RXV3+29 La Palma, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.83943125,-118.048672&key=xxx

Multiple addresses found, the first will be returned:

  4870 Embassy Cir, La Palma, CA 90623, USA

  4862 Crescent Ave, La Palma, CA 90623, USA

  4801-4843 Embassy Cir, La Palma, CA 90623, USA

  La Palma, CA, USA

  La Palma, CA 90623, USA

  Orange County, CA, USA

  California, USA

  United States

  RXQ2+QG La Palma, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.8571850672269,-118.040623134454&key=xxx

Multiple addresses found, the first will be returned:

  5321 Seville Cir, La Palma

  931 Blue Heron, Seal Beach, CA 90740, USA

  Hellman Ranch Trail, Seal Beach, CA 90740, USA

  Seal Beach, CA 90740, USA

  Seal Beach, CA, USA

  Orange County, CA, USA

  California, USA

  United States

  QW24+HP Seal Beach, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.7296670357143,-118.088303785714&key=xxx

Multiple addresses found, the first will be returned:

  119 Surfside Ave., Seal Beach, CA 90740, USA

  90-118 Surfside Ave., Surfside, CA 90743, USA

  HUNTINGTN BCH, CA 92649, USA

  Orange County, CA, USA

  California, USA

  United States

  PWH6+VM Seal Beach, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.7704128314607,-118.086649449438&key=xxx

Multiple addresses found, the first will be returned:

  13260 St Andrews Dr, Seal Beach, CA 90740, USA

  13280 St Andrews Dr, Seal Beach, CA 90740, USA

  1491 Interlachen Rd, Seal Beach, CA 90740, USA

  Oakmont Rd, Seal Beach, CA 90740, USA

  Leisure World, 

Multiple addresses found, the first will be returned:

  Beach-Catherine, Stanton, CA 90680, USA

  12221 Beach Blvd, Stanton, CA 90680, USA

  12221 Beach Blvd, Garden Grove, CA 92841, USA

  12232 Beach Blvd, Stanton, CA 90680, USA

  12298-12142 Beach Blvd, Stanton, CA 90680, USA

  Stanton, CA 90680, USA

  Stanton, CA, USA

  Orange County, CA, USA

  California, USA

  United States

  Q2M4+XV Stanton, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.7878834285714,-117.982881857143&key=xxx

Multiple addresses found, the first will be returned:

  8590 Chapman Ave, Stanton, CA 90680, USA

  8564-8598 Chapman Ave, Garden Grove, CA 92841, USA

  Stanton, CA, USA

  Garden Grove, CA 92841, USA

  Orange County, CA, USA

  California, USA

  United States

  Q2Q8+5R Stanton, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.7775519189189,-117.994422162162&key=xxx

Multiple addresses found, the first will be returned:

  12653 Gl

  1031-1001 Turndell Rd, La Habra, CA 90631, USA

  La Habra, CA 90631, USA

  Orange County, CA, USA

  California, USA

  United States

  W3V5+FM La Habra, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.9412273645833,-117.888501645833&key=xxx

Multiple addresses found, the first will be returned:

  12814 Brea Blvd #125, Brea, CA 92821, USA

  13220-14018 Brea Blvd, Brea, CA 92821, USA

  Brea, CA 92821, USA

  Orange County, CA, USA

  California, USA

  United States

  W4R6+FH Brea, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.923450787234,-117.819829513298&key=xxx

Multiple addresses found, the first will be returned:

  5268 CA-142, Brea, CA 92823, USA

  5635-5143 CA-142, Brea, CA 92823, USA

  Brea, CA 92821, USA

  Orange County, CA, USA

  California, USA

  United States

  W5FJ+93 Brea, CA, USA

Source : https://maps.googleapis.com/maps/api/geocode/json?latlng=33.826840125,-117.961898625&key=xxx

Multiple add

In [1]:
#save.image(file="open_ended.RData")
load(file="open_ended.RData")

## Simplify the graph by removing isolated nodes and aggregating multiple edge

In [4]:
los_angeles_nodes <- V(location_graph)$name[sapply( V(location_graph)$location_id,function (x){grepl( "Los Angeles", x, fixed = TRUE)})]

In [5]:
los_angeles_graph <- induced.subgraph(location_graph,los_angeles_nodes)

In [6]:
sprintf("Los Angeles Nodes %d Edge %d",vcount(los_angeles_graph),ecount(los_angeles_graph))

[1] "Los Angeles Nodes 633 Edge 296254"

In [7]:
cl<-clusters(los_angeles_graph)
largest_g<-induced.subgraph(los_angeles_graph, which(cl$membership == which.max(cl$csize)))

In [8]:
location_graph_simplified <- simplify(largest_g, edge.attr.comb="mean",remove.loops=FALSE)

In [9]:
sprintf("# Nodes: %d , # Edges: %d",vcount(location_graph_simplified),ecount(location_graph_simplified))

[1] "# Nodes: 633 , # Edges: 157698"

## Convert the graph to a directed graph

In [10]:
location_graph_directed <- as.directed(location_graph_simplified,'mutual')

In [11]:
sprintf("# Nodes: %d , # Edges: %d",vcount(location_graph_directed),ecount(location_graph_directed))

[1] "# Nodes: 633 , # Edges: 315396"

## Initialize Graph for Linear Threshold Model

In [12]:
library(stringr)

In [13]:
# invert the edge weight, as we want far distance (high weight) to have little influence
E(location_graph_directed)$weight <- 1/E(location_graph_directed)$weight

In [14]:
# normalize the edge weights, such that the sum of all the in edges is 1
E(location_graph_directed)$weight <- E(location_graph_directed)$weight / strength(location_graph_directed, mode="in")[get.edgelist(location_graph_directed)[,2]]

### linear threshold model algorithm

In [15]:
# select a random amount of nodes for target size to activate... based on heuristics ...
n_initial_nodes <- 10
start_nodes <- names(sort(degree(location_graph_directed),decreasing=TRUE)[1:n_initial_nodes])

In [16]:
# select a random amount of nodes for target size to activate... based on heuristics ...
n_initial_nodes <- 10
start_nodes <- names(sort(degree(location_graph_directed),decreasing=TRUE)[1:n_initial_nodes])
trials <- 50
time_steps <- 25
active_counts <- matrix(nrow=trials,ncol=time_steps)

In [ ]:
for (T in seq_len(trials)){
    print(sprintf("===================Trial %d=================",T))
    flush.console()
    # each node has a latent threshold to become active from influence of neighbors
    V(location_graph_directed)$theta <- runif(vcount(location_graph_directed))
    
    # change the active status to ON!
    V(location_graph_directed)$active <- 0
    V(location_graph_directed)[start_nodes]$active <- 1
    
    # initiate active nodes
    active <- integer(vcount(location_graph_directed))
    names(active) <- V(location_graph_directed)$name
    active[start_nodes] <- 1
    
    # apply algorithm for T time steps
    for (t in seq_len(time_steps)){
        print(sprintf("===================Time Step %d=================",t))
        flush.console()

        # iterate over all nodes to check whether activated or not
        for (node in V(location_graph_directed)$name){
            ptm <- proc.time()

            # find all the "in" incident edges to a node
            incident_edges <- incident(location_graph_directed,v=node,mode='in')

            # get the vertex ids for all the incident edges to a node
            neighbor_nodes_order <- sapply(attributes(incident_edges)$vnames,function(x) {str_split(x,'\\|',simplify=TRUE)[1]})

            # get all the vertices for the in incident edges
            neighbor_nodes <- V(location_graph_directed)[neighbor_nodes_order] # neighbors(location_graph_directed,node,mode='in')

            # check to see if node is now activated or not
            activation_threshold <- get.vertex.attribute(location_graph_directed,'theta',node)
            activate <- as.integer(sum(incident_edges$weight * neighbor_nodes[neighbor_nodes_order]$active) > 
                                   activation_threshold)

            # add to activate vector to then activate all the nodes next round
            active[node] <- activate

    #         print_string <- sprintf("Node %d, elapsed time %.4f",count,(proc.time() - ptm)[1])
    #         print(print_string)
            flush.console()
        }
        new_active_nodes <- names(which(active == 1))
        V(location_graph_directed)[new_active_nodes]$active <- 1
        active_counts[T,t] <- sum(V(location_graph_directed)[new_active_nodes]$active)

    }
}

[1] "===================Trial 1================="
[1] "===================Time Step 1================="
[1] "===================Time Step 2================="
[1] "===================Time Step 3================="
[1] "===================Time Step 4================="
[1] "===================Time Step 5================="
[1] "===================Time Step 6================="
[1] "===================Time Step 7================="
[1] "===================Time Step 8================="
[1] "===================Time Step 9================="
[1] "===================Time Step 10================="
[1] "===================Time Step 11================="
[1] "===================Time Step 12================="
[1] "===================Time Step 13================="
[1] "===================Time Step 14================="
[1] "===================Time Step 15================="
[1] "===================Time Step 16================="
[1] "===================Time Step 17================="
[1] "===================

[1] "===================Time Step 21================="
[1] "===================Time Step 22================="
[1] "===================Time Step 23================="
[1] "===================Time Step 24================="
[1] "===================Time Step 25================="
[1] "===================Trial 7================="
[1] "===================Time Step 1================="
[1] "===================Time Step 2================="
[1] "===================Time Step 3================="
[1] "===================Time Step 4================="
[1] "===================Time Step 5================="
[1] "===================Time Step 6================="
[1] "===================Time Step 7================="
[1] "===================Time Step 8================="
[1] "===================Time Step 9================="
[1] "===================Time Step 10================="
[1] "===================Time Step 11================="
[1] "===================Time Step 12================="
[1] "===================

[1] "===================Time Step 16================="
[1] "===================Time Step 17================="
[1] "===================Time Step 18================="
[1] "===================Time Step 19================="
[1] "===================Time Step 20================="
[1] "===================Time Step 21================="
[1] "===================Time Step 22================="
[1] "===================Time Step 23================="
[1] "===================Time Step 24================="
[1] "===================Time Step 25================="
[1] "===================Trial 13================="
[1] "===================Time Step 1================="
[1] "===================Time Step 2================="
[1] "===================Time Step 3================="
[1] "===================Time Step 4================="
[1] "===================Time Step 5================="
[1] "===================Time Step 6================="
[1] "===================Time Step 7================="
[1] "================

[1] "===================Time Step 11================="
[1] "===================Time Step 12================="
[1] "===================Time Step 13================="
[1] "===================Time Step 14================="
[1] "===================Time Step 15================="
[1] "===================Time Step 16================="
[1] "===================Time Step 17================="
[1] "===================Time Step 18================="
[1] "===================Time Step 19================="
[1] "===================Time Step 20================="
[1] "===================Time Step 21================="
[1] "===================Time Step 22================="
[1] "===================Time Step 23================="
[1] "===================Time Step 24================="
[1] "===================Time Step 25================="
[1] "===================Trial 19================="
[1] "===================Time Step 1================="
[1] "===================Time Step 2================="
[1] "===========